In [ ]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import os
from src.PriceChecker.components.data_ingestion import DataScraper
from src.PriceChecker.components.data_validation import CSVProcessor
from src.PriceChecker.components.data_store import CSVToDatabaseLoader
from src.PriceChecker.constant.database import db_config
from src.PriceChecker.components.visualization import ModelPriceComparer

app = Flask(__name__, template_folder='E:\\assignment\\templates')

def create_directories():
    ingestion_dir = 'artifact/data_ingestion'
    validation_dir = 'artifact/data_validation'
    os.makedirs(ingestion_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/scrape', methods=['POST'])
def scrape():
    query = request.form.get('query')
    
    if not query:
        return jsonify({'error': 'Query parameter missing'}), 400
    
    datainge = DataScraper()

    # Scrape data from both Reliance Digital and Flipkart
    reliance_data = datainge.scrape_reliance(query) or []
    flipkart_data = datainge.scrape_flipkart(query) or []

    if not reliance_data and not flipkart_data:
        return jsonify({'error': 'No data extracted from both sources.'}), 500

    # Create necessary directories
    create_directories()

    # Save the raw scraped data to CSV files
    reliance_df = pd.DataFrame(reliance_data, columns=['Model Name', 'Price'])
    flipkart_df = pd.DataFrame(flipkart_data, columns=['Model Name', 'Price'])
    
    reliance_raw_path = 'artifact/data_ingestion/reliance_data.csv'
    flipkart_raw_path = 'artifact/data_ingestion/flipkart_data.csv'
    
    reliance_df.to_csv(reliance_raw_path, index=False)
    flipkart_df.to_csv(flipkart_raw_path, index=False)

    # Clean the data
    reliance_processor = CSVProcessor(reliance_raw_path, 'artifact/data_validation/reliance_data.csv')
    reliance_processor.clean_csv()

    flipkart_processor = CSVProcessor(flipkart_raw_path, 'artifact/data_validation/flipkart_data.csv')
    flipkart_processor.clean_csv()

    # Load cleaned data to the database
    csv_to_table = {
        'artifact/data_validation/reliance_data.csv': 'reliance_data',
        'artifact/data_validation/flipkart_data.csv': 'flipkart_data'
    }

    loader = CSVToDatabaseLoader(db_config, csv_to_table)
    loader.run()

    # Compare and visualize the prices
    comparer = ModelPriceComparer()
    output_file = 'matched_models_with_prices.csv'
    df_matched = comparer.find_matching_names_and_compare_prices(
        'artifact/data_validation/reliance_data.csv',
        'artifact/data_validation/flipkart_data.csv',
        output_file
    )

    image_path = 'E:\\assignment\\static\\price_comparison.png'
    comparer.visualize_price_differences(df_matched, image_path)

    # Combine results from both sources for display
    combined_data = {
        'Reliance Digital': reliance_data,
        'Flipkart': flipkart_data
    }

    return render_template("results.html", data=combined_data, image_path=image_path)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import os
from src.PriceChecker.components.data_ingestion import DataScraper
from src.PriceChecker.components.data_validation import CSVProcessor
from src.PriceChecker.components.data_store import CSVToDatabaseLoader,db_config
from src.PriceChecker.constant.database import *

app = Flask(__name__, template_folder='E:\\assignment\\templates')

def create_directories():
    ingestion_dir = 'artifact/data_ingestion'
    validation_dir = 'artifact/data_validation'
    os.makedirs(ingestion_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/scrape', methods=['POST'])
def scrape():
    query = request.form.get('query')
    
    if not query:
        return jsonify({'error': 'Query parameter missing'}), 400
    
    datainge = DataScraper()

    # Scrape data from both Reliance Digital and Flipkart
    reliance_data = datainge.scrape_reliance(query) or []
    flipkart_data = datainge.scrape_flipkart(query) or []

    if not reliance_data and not flipkart_data:
        return jsonify({'error': 'No data extracted from both sources.'}), 500

    # Create necessary directories
    create_directories()

    # Save the raw scraped data to CSV files
    reliance_df = pd.DataFrame(reliance_data, columns=['Model Name', 'Price'])
    flipkart_df = pd.DataFrame(flipkart_data, columns=['Model Name', 'Price'])
    
    reliance_raw_path = 'artifact/data_ingestion/reliance_data.csv'
    flipkart_raw_path = 'artifact/data_ingestion/flipkart_data.csv'
    
    reliance_df.to_csv(reliance_raw_path, index=False)
    flipkart_df.to_csv(flipkart_raw_path, index=False)

    # Clean the data
    reliance_processor = CSVProcessor(reliance_raw_path, 'artifact/data_validation/reliance_data.csv')
    reliance_processor.clean_csv()

    flipkart_processor = CSVProcessor(flipkart_raw_path, 'artifact/data_validation/flipkart_data.csv')
    flipkart_processor.clean_csv()

    # Load cleaned data to the database
    csv_to_table = {
        'artifact/data_validation/reliance_data.csv': 'reliance_data',
        'artifact/data_validation/flipkart_data.csv': 'flipkart_data'
    }

    loader = CSVToDatabaseLoader(db_config, csv_to_table)
    loader.run()

    # Combine results from both sources for display
    combined_data = {
        'Reliance Digital': reliance_data,
        'Flipkart': flipkart_data
    }

    return render_template("results.html", data=combined_data)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from flask import Flask, request, jsonify, render_template
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
from src.PriceChecker.components.data_ingestion import DataScraper

app = Flask(__name__, template_folder='E:\\assignment\\templates')

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/scrape', methods=['POST'])
def scrape():
    query = request.form.get('query')
    
    if not query:
        return jsonify({'error': 'Query parameter missing'}), 400
    
    datainge = DataScraper()

    # Scrape data from both Reliance Digital and Flipkart
    reliance_data = datainge.scrape_reliance(query) or []
    flipkart_data = datainge.scrape_flipkart(query) or []

    if not reliance_data and not flipkart_data:
        return jsonify({'error': 'No data extracted from both sources.'}), 500

    # Combine results from both sources
    combined_data = {
        'Reliance Digital': reliance_data,
        'Flipkart': flipkart_data
    }

    return render_template("results.html", data=combined_data)

if __name__ == '__main__':
    app.run(debug=True)
